In [1]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [2]:
# Generating Empty Aggregator to be loaded 

setting = 'FedAvg'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = '/home/ubuntu/FedEM_Shubh/FedEM/weights/cifar_10_50_rnds/FedAvg_adv/'
args_.validation = False
args_.tune_steps = 0

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████| 80/80 [00:00<00:00, 267.09it/s]


===> Initializing clients..


100%|██████████| 80/80 [00:12<00:00,  6.25it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.300 | Train Acc: 10.068% |Test Loss: 2.300 | Test Acc: 10.260% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [3]:
# Import Model Weights
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
#     args_.save_path ='weights/final/femnist/fig1_take3/local_adv/'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/FedAvg_adv/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load(args_.save_path + 'train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
#     args_.save_path = 'weights/final/femnist/figperturb/fedem_avg_p0_1/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/train_client_weights.npy")
    weights = np.load(args_.save_path+"train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/figperturb/fedem_avg_p0_1/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [4]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

In [19]:
new_num_models = len(models_test)
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 4.5

  
batch_size = min(custom_batch_size, dataloader.y_data.shape[0])

t1 = Transferer(models_list=[models_test[0]], dataloader=dataloader)
t1.generate_victims([0]) 

# Perform Attacks
t1.atk_params = PGD_Params()
t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)

t1.generate_advNN(0)
t1.generate_xadv(atk_type = "pgd", load_single_batch=True)

In [5]:
data_dir = '/home/ubuntu/FedEM_Shubh/FedEM/data_grad_analysis'

# torch.save(t1.x_adv, os.path.join(data_dir, 'adv_data_benign_model'))
# torch.save(t1.y_true, os.path.join(data_dir, 'true_labels'))
# torch.save(t1.x_orig, os.path.join(data_dir, 'benign_data_benign_model'))


In [6]:
benign_data_benign_model = torch.load(os.path.join(data_dir, 'benign_data_benign_model'))
adv_data_benign_model = torch.load(os.path.join(data_dir, 'adv_data_benign_model'))
y_true = torch.load(os.path.join(data_dir, 'true_labels'))

benign_data_benign_model.requires_grad = True
adv_data_benign_model.requires_grad = True
# y_true.requires_grad = True

In [10]:
model = models_test[0]
model.train()

out = model(benign_data_benign_model)

criterion = nn.CrossEntropyLoss(reduction="none")
loss = criterion(out, y_true).mean()

for n,p in model.named_parameters():
    if p.grad!=None:
        p.grad = torch.zeros_like(p.grad)

criterion.zero_grad()

# for n,p in model.named_parameters():
#     print(p.grad)
loss.backward()

gradients = {'inputs': adv_data_benign_model.grad}
for n,p in model.named_parameters():
    gradients[n] = p.grad

In [11]:
torch.save(gradients, '/home/ubuntu/FedEM_Shubh/FedEM/gradients_grad_analysis/adv_data_benign_model_grads')

In [73]:
gradients['features.0.0.weight'].shape

torch.Size([32, 3, 3, 3])